## Dicom -> Nifti data conversion

Before making use of the atlas based segmentation tools in this repository, all data should be converted from Dicom to Nifti as a first step. This Jupyter Notebook demonstrates how this can be done.

In [ ]:
import sys
import os
sys.path.append('../../..')

import pydicom

from platipy.dicom.rtstruct_to_nifti.convert import convert_rtstruct

dicom_search_path = "/path/to/dicom/data" # Location where Dicom data is stored
data_path = './data' # Path where converted files should be placed

**The following cell crawls the Dicom search path to find one series per directory**

*Note: this was designed to be run on data that was exported from MIM*

In [ ]:
data_objects = {}

for root, dirs, files in os.walk(dicom_search_path, topdown=False):
    for name in files:
        f = os.path.join(root, name)
        ds = pydicom.read_file(f, force=True)
        
        if not 'Modality' in ds:
            continue
        
        case = ds.PatientID.split(' ')[1]

        if not case in data_objects:
            data_objects[case] = {}
        
        p = f
        if ds.Modality == 'CT':
            p = os.path.dirname(f)
            
        do = {
            'seriesInstanceUID': ds.SeriesInstanceUID, 
            'path': p}
        data_objects[case][ds.Modality] = do
        
        break # Only read one file per dir


**Now that all Dicom data has been found, convert it to Nifti**

In [ ]:
import SimpleITK as sitk

for case in data_objects:
    
    case_dir = os.path.join(data_path, case)
    if not os.path.exists(case_dir):
        os.makedirs(case_dir)
    do = data_objects[case]
   
    # Convert the CT and RT Struct
    ct_path = do['CT']['path']
    rtstruct_path = do['RTSTRUCT']['path']
    convert_rtstruct(ct_path, rtstruct_path, output_img='CT.nii.gz', output_dir=case_dir)
